#유전체 정보 다운로드(인간)

In [1]:
!mkdir -p genome/human
!wget -O genome/human/GRCh38.fna.gz \
 ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000/001/405/GCA_000001405.28_GRCh38.p13/GCA_000001405.28_GRCh38.p13_genomic.fna.gz


--2025-11-19 10:55:28--  ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000/001/405/GCA_000001405.28_GRCh38.p13/GCA_000001405.28_GRCh38.p13_genomic.fna.gz
           => ‘genome/human/GRCh38.fna.gz’
Resolving ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)... 130.14.250.13, 130.14.250.7, 130.14.250.10, ...
Connecting to ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)|130.14.250.13|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /genomes/all/GCA/000/001/405/GCA_000001405.28_GRCh38.p13 ... done.
==> SIZE GCA_000001405.28_GRCh38.p13_genomic.fna.gz ... 964899914
==> PASV ... done.    ==> RETR GCA_000001405.28_GRCh38.p13_genomic.fna.gz ... done.
Length: 964899914 (920M) (unauthoritative)

GCA_000001405.28_GR 100%[===================>] 920.20M  13.5MB/s    in 71s     

2025-11-19 10:56:39 (13.0 MB/s) - ‘genome/human/GRCh38.fna.gz’ saved [964899914]



In [2]:
#압축해제
!gunzip genome/human/GRCh38.fna.gz

#데이터 전처리

In [3]:
!pip install biopython tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 50.1 MB/s eta 0:00:00


In [8]:
import csv
from Bio import SeqIO
from tqdm import tqdm
from itertools import product

# --- 파라미터 ---
INPUT_FILE = '/content/genome/human/GRCh38.fna'
OUTPUT_FILE = '/content/genome/human/preprocessed_windows.csv'

K_MER = 6
SEQ_LEN = 512
STRIDE = 256
MAX_N_RATIO = 0.05

USE_ALL_CHROMOSOMES = True   # 모든 염색체 사용
MAX_CHROMOSOMES = 1          # 테스트용. 실제 학습은 이 라인 삭제 추천.

# --- 1) K-MER vocab 생성 ---
def build_kmer_vocab(k=6):
    bases = ["A", "T", "G", "C"]

    vocab = {
        "[PAD]": 0,
        "[UNK]": 1,
        "[CLS]": 2,
        "[SEP]": 3,
    }

    idx = 4
    for kmer in product(bases, repeat=k):
        vocab["".join(kmer)] = idx
        idx += 1

    return vocab

vocab = build_kmer_vocab(k=K_MER)
unk_id = vocab["[UNK]"]


# --- 2) sequence -> k-mer → token id 변환 ---
def seq_to_token_ids(seq, k=6):
    kmers = [seq[i:i+k] for i in range(len(seq) - k + 1)]
    ids = [vocab.get(k, unk_id) for k in kmers]
    return ids


# --- 3) 메인 전처리 함수 ---
def preprocess_genome_to_csv(input_path, output_path):
    print(f"Processing to CSV: {output_path}")

    count = 0

    with open(output_path, 'w', newline='', encoding='utf-8') as f_out:
        writer = csv.writer(f_out)

        # CSV header
        writer.writerow(["id", "chrom", "start_pos", "token_ids"])

        for record in tqdm(SeqIO.parse(input_path, "fasta"), desc="Chromosomes"):

            if not USE_ALL_CHROMOSOMES and count >= MAX_CHROMOSOMES:
                break

            full_seq = str(record.seq).upper()
            chrom_id = record.id

            if len(full_seq) < SEQ_LEN:
                continue

            # 윈도우 slicing
            for i in range(0, len(full_seq) - SEQ_LEN + 1, STRIDE):
                chunk = full_seq[i:i+SEQ_LEN]

                # N 필터링
                if chunk.count('N') / SEQ_LEN > MAX_N_RATIO:
                    continue

                # k-mer → token id
                token_ids = seq_to_token_ids(chunk, K_MER)
                token_ids_str = " ".join(map(str, token_ids))

                row_id = f"{chrom_id}_{i}"

                writer.writerow([row_id, chrom_id, i, token_ids_str])

            count += 1

    print("\n완료! CSV 저장됨:", output_path)


# 실행
preprocess_genome_to_csv(INPUT_FILE, OUTPUT_FILE)


Processing to CSV: /content/genome/human/preprocessed_windows.csv


Chromosomes: 640it [47:35,  4.46s/it]


완료! CSV 저장됨: /content/genome/human/preprocessed_windows.csv


In [10]:
file_path = "/content/genome/human/GRCh38.fna"

with open(file_path, "r") as f:
    for i in range(5):
        print(f.readline().strip())


>CM000663.2 Homo sapiens chromosome 1, GRCh38 reference primary assembly
NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN
NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN
NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN
NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN


In [9]:
!head -n 5 /content/genome/human/preprocessed_windows.csv

id,chrom,start_pos,token_ids
CM000663.2_9984,CM000663.2,9984,1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1091 257 1016 4052 3911 3347 1091 257 1016 4052 3911 3347 1091 257 1016 4052 3911 3347 1091 257 1016 4052 3911 3347 1091 257 1016 4052 3911 3347 1091 257 1016 4052 3911 3347 1091 257 1016 4052 3911 3347 1091 257 1016 4052 3911 3347 1091 257 1016 4052 3911 3347 1091 257 1016 4052 3911 3347 1091 257 1016 4052 3911 3347 1091 257 1016 4052 3911 3347 1091 257 1016 4052 3911 3347 1091 257 1016 4052 3911 3347 1091 257 1016 4052 3911 3347 1091 257 1016 4052 3911 3347 1091 257 1016 4052 3911 3347 1091 256 1012 4039 3859 3139 257 1016 4052 3911 3347 1091 257 1016 4052 3911 3347 1091 257 1016 4052 3911 3347 1091 257 1016 4052 3911 3347 1091 257 1016 4052 3911 3347 1091 257 1016 4052 3911 3347 1091 259 1025 4088 4052 3911 3347 1091 257 1016 4052 3911 3347 1091 257 1016 4052 3911 3347 1091 257 1016 4052 3911 3347 1091 257 1016 4052 3911 3347 1089 248 980 3911 3347 1091 257 1016 4052 3911 3347 1091 257 1016 

fna파일은 여러개의 블록으로 이루어짐. 이때 이 블록을 레코드라고 함.

MAX_CHROMOSOMES = 1  << 이 코드는 레코드 첫 값만 불러오는 함수. 첫번째만 불러오는 이유는 3.1Gb를 돌리면 너무 오래걸리고 코랩은 런타임이 멈출 수 있어서임

첫번째 레코드만 불러와도 이걸 기준으로 나머지를 불러오기 때문에 약간의 차이는 있을 수 있지만 크게 영향을 주진 않는다고 해서 사용했습니다. 만약 모델의 성능을 더 올릴라면 이걸 없애고 모든 레코드값을 불러오는 식으로 하면 됨


N = 결측값입니다.
여기서 fna파일을 한번에 전처리 했습니다.

맨 처음 레코드를 읽고 불러 온 다음 ATGCACGT << 이런 형태로 6개 연속 염기로 바꾸어 주고 이걸 정수 아이디(Ex.88 921 23)로 바꿈


이 데이터를 통해 모델을 만들면 됨